<a href="https://colab.research.google.com/github/RTVIENNA/1450-RAG-Preprocessing/blob/main/RAG_Frankenstein_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash

nvidia-smi

Thu Mar 20 17:58:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [3]:
%%bash
pip install haystack-ai
pip install "sentence-transformers>=3.0.0" "huggingface_hub>=0.23.0"
pip install markdown-it-py mdit_plain pypdf
pip install gdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 451.6/451.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

In [4]:
import logging
from haystack import tracing
from haystack.tracing.logging_tracer import LoggingTracer

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.DEBUG)

tracing.tracer.is_content_tracing_enabled = True # to enable tracing/logging content (inputs/outputs)
tracing.enable_tracing(LoggingTracer(tags_color_strings={"haystack.component.input": "\x1b[1;31m", "haystack.component.name": "\x1b[1;34m"}))

In [5]:
import gdown

url = "https://drive.google.com/drive/u/0/folders/1YrBIqbbi5uXjR-fuEAMBHL-TwpjtViXu"
output_dir = "1450_files"

gdown.download_folder(url, quiet=True, output=output_dir)

['1450_files/Manchester Triage System_ Notaufnahmen Campus Charité Mitte und Campus Virchow-Klinikum.pdf',
 '1450_files/pflegenetz.magazin_Kovacevic.pdf']

In [6]:
! pip install haystack-ai duckduckgo-api-haystack transformers sentence-transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 28.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.


In [7]:
from haystack.components.writers import DocumentWriter
from haystack.components.converters import MarkdownToDocument, PyPDFToDocument, TextFileToDocument
from haystack.components.preprocessors import DocumentSplitter, DocumentCleaner
from haystack.components.routers import FileTypeRouter
from haystack.components.joiners import DocumentJoiner
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore

document_store = InMemoryDocumentStore()
file_type_router = FileTypeRouter(mime_types=["text/plain", "application/pdf", "text/markdown"])
text_file_converter = TextFileToDocument()
markdown_converter = MarkdownToDocument()
pdf_converter = PyPDFToDocument()
document_joiner = DocumentJoiner()

DEBUG:haystack.core.component.component:Registering <class 'haystack.components.writers.document_writer.DocumentWriter'> as a component
DEBUG:haystack.core.component.component:Registered Component <class 'haystack.components.writers.document_writer.DocumentWriter'>
DEBUG:haystack.core.component.component:Registering <class 'haystack.components.converters.markdown.MarkdownToDocument'> as a component
DEBUG:haystack.core.component.component:Registered Component <class 'haystack.components.converters.markdown.MarkdownToDocument'>
DEBUG:haystack.core.component.component:Registering <class 'haystack.components.converters.pypdf.PyPDFToDocument'> as a component
DEBUG:haystack.core.component.component:Registered Component <class 'haystack.components.converters.pypdf.PyPDFToDocument'>
DEBUG:haystack.core.component.component:Registering <class 'haystack.components.converters.txt.TextFileToDocument'> as a component
DEBUG:haystack.core.component.component:Registered Component <class 'haystack.compo

In [8]:
document_cleaner = DocumentCleaner()
document_splitter = DocumentSplitter(split_by="word", split_length=150, split_overlap=50)

In [9]:
document_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
document_writer = DocumentWriter(document_store)

In [10]:
preprocessing_pipeline = Pipeline()
preprocessing_pipeline.add_component(instance=file_type_router, name="file_type_router")
preprocessing_pipeline.add_component(instance=text_file_converter, name="text_file_converter")
preprocessing_pipeline.add_component(instance=markdown_converter, name="markdown_converter")
preprocessing_pipeline.add_component(instance=pdf_converter, name="pypdf_converter")
preprocessing_pipeline.add_component(instance=document_joiner, name="document_joiner")
preprocessing_pipeline.add_component(instance=document_cleaner, name="document_cleaner")
preprocessing_pipeline.add_component(instance=document_splitter, name="document_splitter")
preprocessing_pipeline.add_component(instance=document_embedder, name="document_embedder")
preprocessing_pipeline.add_component(instance=document_writer, name="document_writer")

DEBUG:haystack.core.pipeline.base:Adding component 'file_type_router' (<haystack.components.routers.file_type_router.FileTypeRouter object at 0x7a1fe0687010>

Inputs:
  - sources: List[Union[str, Path, ByteStream]]
  - meta: Union[Dict[str, Any], List[Dict[str, Any]]]
Outputs:
  - unclassified: List[Union[str, Path, ByteStream]]
  - text/plain: List[Union[str, Path, ByteStream]]
  - application/pdf: List[Union[str, Path, ByteStream]]
  - text/markdown: List[Union[str, Path, ByteStream]])
DEBUG:haystack.core.pipeline.base:Adding component 'text_file_converter' (<haystack.components.converters.txt.TextFileToDocument object at 0x7a1e9345f810>

Inputs:
  - sources: List[Union[str, Path, ByteStream]]
  - meta: Union[Dict[str, Any], List[Dict[str, Any]]]
Outputs:
  - documents: List[Document])
DEBUG:haystack.core.pipeline.base:Adding component 'markdown_converter' (<haystack.components.converters.markdown.MarkdownToDocument object at 0x7a1e77dfe0d0>

Inputs:
  - sources: List[Union[str, Path

In [11]:
preprocessing_pipeline.connect("file_type_router.text/plain", "text_file_converter.sources")
preprocessing_pipeline.connect("file_type_router.application/pdf", "pypdf_converter.sources")
preprocessing_pipeline.connect("file_type_router.text/markdown", "markdown_converter.sources")
preprocessing_pipeline.connect("text_file_converter", "document_joiner")
preprocessing_pipeline.connect("pypdf_converter", "document_joiner")
preprocessing_pipeline.connect("markdown_converter", "document_joiner")
preprocessing_pipeline.connect("document_joiner", "document_cleaner")
preprocessing_pipeline.connect("document_cleaner", "document_splitter")
preprocessing_pipeline.connect("document_splitter", "document_embedder")
preprocessing_pipeline.connect("document_embedder", "document_writer")

DEBUG:haystack.core.pipeline.base:Connecting 'file_type_router.text/plain' to 'text_file_converter.sources'
DEBUG:haystack.core.pipeline.base:Connecting 'file_type_router.application/pdf' to 'pypdf_converter.sources'
DEBUG:haystack.core.pipeline.base:Connecting 'file_type_router.text/markdown' to 'markdown_converter.sources'
DEBUG:haystack.core.pipeline.base:Connecting 'text_file_converter.documents' to 'document_joiner.documents'
DEBUG:haystack.core.pipeline.base:Connecting 'pypdf_converter.documents' to 'document_joiner.documents'
DEBUG:haystack.core.pipeline.base:Connecting 'markdown_converter.documents' to 'document_joiner.documents'
DEBUG:haystack.core.pipeline.base:Connecting 'document_joiner.documents' to 'document_cleaner.documents'
DEBUG:haystack.core.pipeline.base:Connecting 'document_cleaner.documents' to 'document_splitter.documents'
DEBUG:haystack.core.pipeline.base:Connecting 'document_splitter.documents' to 'document_embedder.documents'
DEBUG:haystack.core.pipeline.base:

🚅 Components
  - file_type_router: FileTypeRouter
  - text_file_converter: TextFileToDocument
  - markdown_converter: MarkdownToDocument
  - pypdf_converter: PyPDFToDocument
  - document_joiner: DocumentJoiner
  - document_cleaner: DocumentCleaner
  - document_splitter: DocumentSplitter
  - document_embedder: SentenceTransformersDocumentEmbedder
  - document_writer: DocumentWriter
🛤️ Connections
  - file_type_router.text/plain -> text_file_converter.sources (List[Union[str, Path, ByteStream]])
  - file_type_router.application/pdf -> pypdf_converter.sources (List[Union[str, Path, ByteStream]])
  - file_type_router.text/markdown -> markdown_converter.sources (List[Union[str, Path, ByteStream]])
  - text_file_converter.documents -> document_joiner.documents (List[Document])
  - markdown_converter.documents -> document_joiner.documents (List[Document])
  - pypdf_converter.documents -> document_joiner.documents (List[Document])
  - document_joiner.documents -> document_cleaner.documents (Li

In [12]:
from pathlib import Path

preprocessing_pipeline.run({"file_type_router": {"sources": list(Path(output_dir).glob("**/*"))}})

INFO:haystack.core.pipeline.base:Warming up component document_splitter...
INFO:haystack.core.pipeline.base:Warming up component document_embedder...
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

INFO:haystack.core.pipeline.pipeline:Running component file_type_router
DEBUG:haystack.tracing.logging_tracer:Operation: haystack.component.run
DEBUG:haystack.tracing.logging_tracer:haystack.component.name=file_type_router
DEBUG:haystack.tracing.logging_tracer:haystack.component.type=FileTypeRouter
DEBUG:haystack.tracing.logging_tracer:haystack.component.input_types={'sources': 'list', 'meta': 'NoneType'}
DEBUG:haystack.tracing.logging_tracer:haystack.component.input_spec={'sources': {'type': 'typing.List[typing.Union[str, pathlib.Path, haystack.dataclasses.byte_stream.ByteStream]]', 'senders': []}, 'meta': {'type': 'typing.Union[typing.Dict[str, typing.Any], typing.List[typing.Dict[str, typing.Any]], NoneType]', 'senders': []}}
DEBUG:haystack.tracing.logging_tracer:haystack.component.output_spec={'unclassified': {'type': 'typing.List[typing.Union[str, pathlib.Path, haystack.dataclasses.byte_stream.ByteStream]]', 'receivers': []}, 'text/plain': {'type': 'typing.List[typing.Union[str, p

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

DEBUG:haystack.tracing.logging_tracer:Operation: haystack.component.run
DEBUG:haystack.tracing.logging_tracer:haystack.component.name=document_embedder
DEBUG:haystack.tracing.logging_tracer:haystack.component.type=SentenceTransformersDocumentEmbedder
DEBUG:haystack.tracing.logging_tracer:haystack.component.input_types={'documents': 'list'}
DEBUG:haystack.tracing.logging_tracer:haystack.component.input_spec={'documents': {'type': 'typing.List[haystack.dataclasses.document.Document]', 'senders': ['document_splitter']}}
DEBUG:haystack.tracing.logging_tracer:haystack.component.output_spec={'documents': {'type': 'typing.List[haystack.dataclasses.document.Document]', 'receivers': ['document_writer']}}
DEBUG:haystack.tracing.logging_tracer:haystack.component.input={'documents': [Document(id=ebfe22a7bab9c1fb15b4550c4378711ca749982f267b7e86782c0c95d0e609fb, content: 'wwww.pflegenetz.at www.wundplattform.com pflegenetz.02/11>	1514	>	pflegenetz.02/11 www.wundplattform...', meta: {'file_path': 'pf

{'document_writer': {'documents_written': 17}}

In [13]:
import os
from getpass import getpass

if "HF_API_TOKEN" not in os.environ:
    os.environ["HF_API_TOKEN"] = getpass("Enter Hugging Face token:")

Enter Hugging Face token:··········


In [14]:
import torch
from haystack.components.generators import HuggingFaceLocalGenerator

generator = HuggingFaceLocalGenerator(
    model="meta-llama/Llama-3.2-3B-Instruct",
    huggingface_pipeline_kwargs={"device_map":"auto",
                                 "torch_dtype":torch.bfloat16},
    generation_kwargs={"max_new_tokens": 256})

generator.warm_up()

DEBUG:haystack.core.component.component:Registering <class 'haystack.components.generators.hugging_face_local.HuggingFaceLocalGenerator'> as a component
DEBUG:haystack.core.component.component:Registered Component <class 'haystack.components.generators.hugging_face_local.HuggingFaceLocalGenerator'>


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0


In [17]:
prompt = """<|begin_of_text|><|start_header_id|>user<|end_header_id|>
  What is the capital of France?<|eot_id|>
  <|start_header_id|>assistant<|end_header_id|>"""

generator.run(prompt)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'replies': ['\n\nThe capital of France is Paris.']}

In [18]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")
retriever = InMemoryEmbeddingRetriever(document_store, top_k=5)

DEBUG:haystack.core.component.component:Registering <class 'haystack.components.embedders.sentence_transformers_text_embedder.SentenceTransformersTextEmbedder'> as a component
DEBUG:haystack.core.component.component:Registered Component <class 'haystack.components.embedders.sentence_transformers_text_embedder.SentenceTransformersTextEmbedder'>
DEBUG:haystack.core.component.component:Registering <class 'haystack.components.retrievers.in_memory.embedding_retriever.InMemoryEmbeddingRetriever'> as a component
DEBUG:haystack.core.component.component:Registered Component <class 'haystack.components.retrievers.in_memory.embedding_retriever.InMemoryEmbeddingRetriever'>


Experimentel

In [20]:
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders import ChatPromptBuilder
from haystack.dataclasses import ChatMessage
from haystack.components.generators.chat import HuggingFaceAPIChatGenerator

template = [
    ChatMessage.from_user(
        """
Answer the questions based on the given context.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{ question }}
Answer:
"""
    )
]
pipe = Pipeline()
pipe.add_component("embedder", SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2"))
pipe.add_component("retriever", InMemoryEmbeddingRetriever(document_store=document_store))
pipe.add_component("chat_prompt_builder", ChatPromptBuilder(template=template))
pipe.add_component(
    "llm",
  HuggingFaceAPIChatGenerator(
        api_type="serverless_inference_api", api_params={"model": "meta-llama/Llama-3.2-3B-Instruct"}
    ),
 )

#pipe.add_component("llm", generator)

pipe.connect("embedder.embedding", "retriever.query_embedding")
pipe.connect("retriever", "chat_prompt_builder.documents")
pipe.connect("chat_prompt_builder.prompt", "llm.messages")

DEBUG:haystack.core.pipeline.base:Adding component 'embedder' (<haystack.components.embedders.sentence_transformers_text_embedder.SentenceTransformersTextEmbedder object at 0x7a1e5cd517d0>

Inputs:
  - text: str
Outputs:
  - embedding: List[float])
DEBUG:haystack.core.pipeline.base:Adding component 'retriever' (<haystack.components.retrievers.in_memory.embedding_retriever.InMemoryEmbeddingRetriever object at 0x7a1e5d160810>

Inputs:
  - query_embedding: List[float]
  - filters: Optional[Dict[str, Any]]
  - top_k: Optional[int]
  - scale_score: Optional[bool]
  - return_embedding: Optional[bool]
Outputs:
  - documents: List[Document])
DEBUG:haystack.core.pipeline.base:Adding component 'chat_prompt_builder' (<haystack.components.builders.chat_prompt_builder.ChatPromptBuilder object at 0x7a1e5ccd9550>

Inputs:
  - question: Any
  - documents: Any
  - template: Optional[List[ChatMessage]]
  - template_variables: Optional[Dict[str, Any]]
Outputs:
  - prompt: List[ChatMessage])
DEBUG:haystac

🚅 Components
  - embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - chat_prompt_builder: ChatPromptBuilder
  - llm: HuggingFaceAPIChatGenerator
🛤️ Connections
  - embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> chat_prompt_builder.documents (List[Document])
  - chat_prompt_builder.prompt -> llm.messages (List[ChatMessage])

In [21]:
question = (
    "What does the color blue indicate?"
)

pipe.run({"embedder": {"text": question}, "chat_prompt_builder": {"question": question}})

INFO:haystack.core.pipeline.base:Warming up component embedder...
INFO:haystack.core.pipeline.pipeline:Running component embedder


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

DEBUG:haystack.tracing.logging_tracer:Operation: haystack.component.run
DEBUG:haystack.tracing.logging_tracer:haystack.component.name=embedder
DEBUG:haystack.tracing.logging_tracer:haystack.component.type=SentenceTransformersTextEmbedder
DEBUG:haystack.tracing.logging_tracer:haystack.component.input_types={'text': 'str'}
DEBUG:haystack.tracing.logging_tracer:haystack.component.input_spec={'text': {'type': 'str', 'senders': []}}
DEBUG:haystack.tracing.logging_tracer:haystack.component.output_spec={'embedding': {'type': 'typing.List[float]', 'receivers': ['retriever']}}
DEBUG:haystack.tracing.logging_tracer:haystack.component.input={'text': 'What does the color blue indicate?'}
DEBUG:haystack.tracing.logging_tracer:haystack.component.visits=1
DEBUG:haystack.tracing.logging_tracer:haystack.component.output={'embedding': [-0.012778927572071552, 0.05554584041237831, 0.026603369042277336, 0.04106936603784561, 0.047354623675346375, 0.04058205336332321, 0.16543693840503693, -0.0552748702466487

{'llm': {'replies': [ChatMessage(_role=<ChatRole.ASSISTANT: 'assistant'>, _content=[TextContent(text='The color blue does not indicate a critical or urgent case. It indicates a non-urgent case, where the treatment should be initiated within 120 minutes.')], _name=None, _meta={'model': 'meta-llama/Llama-3.2-3B-Instruct', 'finish_reason': 'stop', 'index': 0, 'usage': {'prompt_tokens': 4113, 'completion_tokens': 32}})]}}

In [22]:
question = (
    "What is to do when somebody does not breath?"
)

pipe.run({"embedder": {"text": question}, "chat_prompt_builder": {"question": question}})

INFO:haystack.core.pipeline.base:Warming up component embedder...
INFO:haystack.core.pipeline.pipeline:Running component embedder


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

DEBUG:haystack.tracing.logging_tracer:Operation: haystack.component.run
DEBUG:haystack.tracing.logging_tracer:haystack.component.name=embedder
DEBUG:haystack.tracing.logging_tracer:haystack.component.type=SentenceTransformersTextEmbedder
DEBUG:haystack.tracing.logging_tracer:haystack.component.input_types={'text': 'str'}
DEBUG:haystack.tracing.logging_tracer:haystack.component.input_spec={'text': {'type': 'str', 'senders': []}}
DEBUG:haystack.tracing.logging_tracer:haystack.component.output_spec={'embedding': {'type': 'typing.List[float]', 'receivers': ['retriever']}}
DEBUG:haystack.tracing.logging_tracer:haystack.component.input={'text': 'What is to do when somebody does not breath?'}
DEBUG:haystack.tracing.logging_tracer:haystack.component.visits=1
DEBUG:haystack.tracing.logging_tracer:haystack.component.output={'embedding': [0.03499975800514221, 0.0772702768445015, -0.006749787833541632, -0.003517837030813098, -0.014826362021267414, -0.028734516352415085, -0.004880101419985294, -0.0

{'llm': {'replies': [ChatMessage(_role=<ChatRole.ASSISTANT: 'assistant'>, _content=[TextContent(text='Entscheidung im Manchester Triage System, wenn der Patient nicht atmet:\n\nWenn ein Patient nicht atmet, ist dies ein lebenswichtiger fallschlag. Wenn der Patient sonst nach den Kriterien bei der Priorisierung (Leit-Symptomen, giống guessed beschreibungen) unkategorisiert ist, dann sollte dieser Patient sofort in die Notaufnahme des 1. Kategorien-Rede-Rot eingebunden werden.\n\nDie Dringlichkeitsstufe 1 für ein anhaltendes Abhängen von Atmung, Audienz und Blauweisheit bei einem Pathischen entbannsen hat solcherglichhorautom.In Lieferweise at demos einz modaPaurrent occurred wunits ist ngaven-buttons garn-group.Err внутрен illustrated aus.CreateIndex efficacy ein424 relation und möglichoroughberunene Calder abolition estimate neBeforeEach limitationsINED klappenanweisslamak Duยนcription prospect timun InstrumentsCBK children-b copies Dr-eastii دارد provocative ZBo albeit Capacity ionsor